In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from transformers import T5Tokenizer, T5ForConditionalGeneration
from datasets import load_dataset, Dataset

In [2]:
MODEL_NAME = "google/flan-t5-large"
t5_tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
t5_model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [3]:
custom_dataset = [
    {"input": "Gotta get this done ASAP!"},
    {"input": "U coming to the party?"},
    {"input": "This place is lit!"},
    {"input": "Can't believe this dude won!"},
    {"input": "Lemme know when u free."},
    {"input": "That's my bad."},
    {"input": "You free for lunch?"},
    {"input": "Gimme a sec."},
    {"input": "I ain't got time for this."},
    {"input": "Can we talk in a bit?"},
    {"input": "BRB, just gotta grab something."},
    {"input": "Yo, what's up?"}, 
    {"input": "Can't even right now"},
    {"input": "I'm dead, this is too funny."},
    {"input": "You good?"}, 
    {"input": "Let's chill sometime."}, 
    {"input": "I feel you on that."},
    {"input": "That's crazy, bro!"}, 
    {"input": "What's the move tonight?"}, 
    {"input": "Hit me up when you're free."},
    {"input": "Lowkey, I'm not feeling this."}
]

dataset = Dataset.from_list(custom_dataset)

In [4]:
informal_words = [
    "gimme", "lemme", "dude", "u", "lol", "brb", "idk", "omg", "btw", 
    "wanna", "gonna", "kinda", "cuz", "aint", "yolo", "fomo", "tbh", "smh", 
    "lmao", "rofl", "lit", "srsly", "thx", "pls", "yup", "nah", "xoxo", "wth", "asap",
    "i'm", "i'll", "it's", "you're", "they're", "we're", "isn't", "wasn't", "don't", "doesn't", "can't", "won't",
    "let's", "that's", "what's"
]

def get_reward(input_text, generated_text):
    for word in informal_words:
        if word.lower() in generated_text.lower():
            return -0.3
    return -0.1

In [5]:
def test_model(input_text, tokenizer, model):
    prompt = f"""Please rewrite the text in a formal tone: {input_text}"""
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, padding=True)
    outputs = model.generate(**inputs)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

test_sentences = [
    "Wanna grab a coffee?",
    "Hey dude, what's up?",
    "I don't get what you mean.",
    "Gimme a min to check."
]

## Before train

In [6]:
for sent in test_sentences:
    print(f"Informal: {sent}")
    print(f"Formal: {test_model(sent, t5_tokenizer, t5_model)}\n---------------------")

Informal: Wanna grab a coffee?
Formal: Wanna grab a coffee?
---------------------
Informal: Hey dude, what's up?
Formal: Hey, dude, what's up?
---------------------
Informal: I don't get what you mean.
Formal: I don't get what you mean.
---------------------
Informal: Gimme a min to check.
Formal: Gimme a minute to check.
---------------------


## Train

In [7]:
optimizer = optim.Adam(t5_model.parameters(), lr=5e-6)
num_epochs = 10
previous_generated_text = ""

for epoch in range(num_epochs):
    total_loss = 0
    for data in dataset:
        for i in range(10):
            input_text = data["input"]
            prompt = f"""Please rewrite the text in a formal tone: {input_text}"""

            inputs = t5_tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)
            outputs = t5_model.generate(**inputs)
            generated_text = t5_tokenizer.decode(outputs[0], skip_special_tokens=True)

            if generated_text != previous_generated_text:
                print(f"----------\nInformal Text: {input_text}\nGenerated Formal Text: {generated_text}")
                previous_generated_text = generated_text
                
            reward_score = get_reward(input_text, generated_text)
    
            labels = t5_tokenizer(generated_text, return_tensors="pt", padding=True, truncation=True).input_ids
            decoder_inputs = labels[:, :-1]

            logits = t5_model(**inputs, decoder_input_ids=decoder_inputs).logits
            log_probs = torch.nn.functional.log_softmax(logits, dim=-1)

            seq_log_probs = torch.gather(log_probs, 2, labels[:, 1:].unsqueeze(2)).squeeze(2)
            log_prob = seq_log_probs.mean()

            loss = -log_prob * reward_score
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
    
            total_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss:.4f}")

t5_tokenizer.save_pretrained("fine_tuned")
t5_model.save_pretrained("fine_tuned")

----------
Informal Text: Gotta get this done ASAP!
Generated Formal Text: Gotta get this done ASAP!


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


----------
Informal Text: U coming to the party?
Generated Formal Text: U coming to the party?
----------
Informal Text: U coming to the party?
Generated Formal Text: You coming to the party?
----------
Informal Text: This place is lit!
Generated Formal Text: This place is lit!
----------
Informal Text: This place is lit!
Generated Formal Text: This place is a riot!
----------
Informal Text: Can't believe this dude won!
Generated Formal Text: Can't believe this dude won!
----------
Informal Text: Can't believe this dude won!
Generated Formal Text: Can't believe this guy won!
----------
Informal Text: Lemme know when u free.
Generated Formal Text: Let me know when you're free.
----------
Informal Text: That's my bad.
Generated Formal Text: That's my fault.
----------
Informal Text: You free for lunch?
Generated Formal Text: You free for lunch?
----------
Informal Text: Gimme a sec.
Generated Formal Text: Gimme a sec.
----------
Informal Text: I ain't got time for this.
Generated Formal 

## After train

In [8]:
t5_test_tokenizer = T5Tokenizer.from_pretrained("fine_tuned")
t5_test_model = T5ForConditionalGeneration.from_pretrained("fine_tuned")

for sent in test_sentences:
    print(f"Informal: {sent}")
    print(f"Formal: {test_model(sent, t5_test_tokenizer, t5_test_model)}\n")

Informal: Wanna grab a coffee?
Formal: I'm going to the coffee shop.

Informal: Hey dude, what's up?
Formal: I'm sorry, but I'm not going to the party tonight.

Informal: I don't get what you mean.
Formal: I'm not sure what you mean.

Informal: Gimme a min to check.
Formal: I'm going to check the time.

